In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import general_utils as Ugen
import matplotlib.pyplot as plt
import pandas as pd
import math
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob, os

In [3]:
##GET DATA
#GET FILES
main_dir=os.getcwd()
allFiles = glob.glob(main_dir.split('/Studies')[0]+'/toronto_parking_data'+ "/*.csv")

#LOAD TO DATAFRAME
list_ = []
for file_path in allFiles:
    try:
        df = pd.read_csv(file_path,index_col=None,quoting=2,error_bad_lines=False) #quoting=2
        list_.append(df)
        print ('successfully loaded: '+ file_path)
    except Exception as err:
        if 'EOF following escape character' in str(err):
            try:
                df = pd.read_csv(file_path,index_col=None,encoding='utf-16',error_bad_lines=False) #quoting=2
                list_.append(df)
                print ('successfully loaded: '+ file_path)
            except Exception as err:
                print ('NOT loaded: '+ file_path)
                print (str(err))
        else:
            print ('NOT loaded: '+ file_path)
            print (str(err))
tpt_df = pd.concat(list_)
print (tpt_df.shape)
tpt_df.head()

b'Skipping line 181770: expected 11 fields, saw 12\n'
b'Skipping line 170864: expected 11 fields, saw 12\n'

successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_data_2008.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_data_2009.csv


b'Skipping line 1075521: expected 11 fields, saw 14\n'
b'Skipping line 1413505: expected 11 fields, saw 16\n'


successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_data_2010.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_data_2011.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2012.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2013.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_3.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_4.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/P

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***63611,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,364 EAST MALL,NaN,NaN,ON
1,***99026,20080101,9,STOP HWY PROHIBITED TIME/DAY,60,NaN,NaN,80 RICHMOND ST W,NaN,NaN,ON
2,***63613,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,364 EAST MALL,NaN,NaN,ON
3,***63617,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,362 EAST MALL,NaN,NaN,ON
4,***63608,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,366 EAST MALL,NaN,NaN,ON


In [8]:
tpt_df=tpt_df[pd.notnull(tpt_df['location2'])]
tpt_df=tpt_df.drop(['tag_number_masked','location3','location4','province'],axis=1)
print (tpt_df.shape)

##19 million parking ticket records.....lol


(19412524, 7)


In [29]:
locations=tpt_df['location2'].unique()
len(locations)

568146

In [10]:
len(locations)/2500 ##number of days it will take to get the data with free google api (2500 req/day)

227.2584

So obviously we can't get lat/lon for all locations...let's do it for the top X grossing spots...

In [22]:
tpt_df['year']=tpt_df['date_of_infraction'].map(lambda x: int(str(x)[0:4])) #
tpt_df.head()

,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,year
0,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,364 EAST MALL,2008
1,20080101,9,STOP HWY PROHIBITED TIME/DAY,60,NaN,NaN,80 RICHMOND ST W,2008
2,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,364 EAST MALL,2008
3,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,362 EAST MALL,2008
4,20080101,3,PARK/LEAVE ON PRIVATE PROPERTY,30,NaN,NaN,366 EAST MALL,2008


In [42]:
infraction_ids=list(tpt_df.infraction_code.unique())

In [43]:
tpt_grouped_df=pd.DataFrame({'count' : tpt_df.groupby( ['year','infraction_code','location2'] ).size()}).reset_index()

In [44]:
print(tpt_grouped_df.shape)
tpt_grouped_df.head()
# tpt_grouped_df[tpt_grouped_df['year']==2015]

(2833240, 4)


,year,infraction_code,location2,count
0,2008,1,1 FOUR WINDS DR,1
1,2008,1,1 9 MOBERLY AVE,1
2,2008,1,1 BANGOR RD,2
3,2008,1,1 BEAMAN RD,2
4,2008,1,1 BEDFORD PARK AVE,8


In [47]:
##now calculate total amount...
##create dict of infraction_code:fine_amount
fine_amount_dict={ic:{'fine_amt':tpt_df[tpt_df['infraction_code']==ic].set_fine_amount.median(),
                      'fine_descp':tpt_df[tpt_df['infraction_code']==ic].infraction_description.iloc[0]}    
                          for ic in infraction_ids}

def calc_fine_sum(df):
    return df['count']*fine_amount_dict[df['infraction_code']]['fine_amt']

def get_fine_descrp(df):
    return fine_amount_dict[df['infraction_code']]['fine_descp']


tpt_grouped_df['fine_sum']=tpt_grouped_df.apply(calc_fine_sum,axis=1)
tpt_grouped_df['fine_descrp']=tpt_grouped_df.apply(get_fine_descrp,axis=1)
tpt_grouped_df.sort_values('fine_sum',ascending=False)

,year,infraction_code,location2,count,fine_sum,fine_descrp
2779875,2015,347,1090 DON MILLS RD,1889,472250,PARK IN A FIRE ROUTE
2426803,2014,367,410 COLLEGE ST,742,333900,STAND ON-ST DISABL LDG NO PRMT
1883194,2012,367,410 COLLEGE ST,672,302400,STAND ON-ST DISABL LDG NO PRMT
818428,2010,3,1750 FINCH AVE E,9574,287220,PARK/LEAVE ON PRIVATE PROPERTY
2784954,2015,367,410 COLLEGE ST,604,271800,STAND ON-ST DISABL LDG NO PRMT
450759,2009,3,1750 FINCH AVE E,8669,260070,PARK/LEAVE ON PRIVATE PROPERTY
2421668,2014,347,1090 DON MILLS RD,1028,257000,PARK IN A FIRE ROUTE
53976,2008,3,25 THE WEST MALL,8291,248730,PARK/LEAVE ON PRIVATE PROPERTY
2408008,2014,257,1 BRIMLEY RD S,2296,241080,PARK CONTRARY POSTED CONDITION
2768535,2015,257,1 BRIMLEY RD S,2267,238035,PARK CONTRARY POSTED CONDITION


In [56]:
fine_amount_dict[347]

{'fine_amt': 250.0, 'fine_descp': 'PARK IN A FIRE ROUTE'}

This is wild...so in 2015 there were 1889 parking tickets issued at one location (1090 Don Mills Rd).

That's just over 5 tickets a day on average...and that's if you assume every day at the mall/this location has the same traffic..which is probably not true. In reality, this would be concentrated on weekends so it could be something like 10-20 tickets (random guess) on weekends.

If you assume each ticketed vehicle is there for at least an hour...that's a good chunk of the day where the fire route is at least partially blocked..obviously this needs to be treated more as an actual safety concern and less as an incredible revenue generating source for Toronto's municipal government. 

Obviously this rule is not clear...I don't think people would risk a $250 ticket just to get a closer parking spot..this spot (and likely many others) clearly needs better labelling. Putting a sign on the wall is one thing...how about the government takes $20 from the $472,250 they earned in 2015 alone (okay, maybe 50 percent of the people fought their tickets or didn't pay them...only $236,125) and puts some red paint on the pavement. 




In [58]:
[{infrac_type:infrac_dict['fine_descp']} for infrac_type,infrac_dict in fine_amount_dict.items() if infrac_dict['fine_amt']==0]

[{130: 'STOP PROH TIME MUTUAL STREET'},
 {150: 'PARK VEH BEYOND DESIG AREA'},
 {166: 'PARK ON TRINITY DRIVE'},
 {197: 'PARK NOT WITHIN PERMITTED TIME'},
 {251: 'LEAVE IN PARK NOT DESIG AREA'},
 {252: 'PARK IN PARK - PROHIBITED AREA'},
 {256: 'LEAVE OVERNIGHT WITHOUT PERMIT'},
 {258: 'STOP CONTRARY POSTED CONDITION'},
 {348: 'PARK IN DISABLED NO PERMIT'}]

In [55]:
tpt_df[(tpt_df['infraction_code']==348) & (tpt_df['year']==2008)]

,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,year,year2
541819,20080314,348,PARK IN DISABLED NO PERMIT,450,54,NR,259 SUMACH ST,2008,0
541827,20080314,348,PARK IN DISABLED NO PERMIT,450,55,NR,250 SUMACH ST,2008,0
541850,20080314,348,PARK IN DISABLED NO PERMIT,450,100,NR,295 SACKVILLE ST,2008,0
543826,20080314,348,PARK IN DISABLED NO PERMIT,450,845,AT,1799 ST CLAIR AVE W,2008,0
544495,20080314,348,PARK IN DISABLED NO PERMIT,450,1022,AT,1050 THE QUEENSWAY,2008,0
545496,20080314,348,PARK IN DISABLED NO PERMIT,450,1200,AT,464 DANFORTH AVE,2008,0
546027,20080314,348,PARK IN DISABLED NO PERMIT,450,1237,AT,1995 WESTON RD,2008,0
546321,20080314,348,PARK IN DISABLED NO PERMIT,450,1303,AT,1966 WESTON RD,2008,0
547188,20080314,348,PARK IN DISABLED NO PERMIT,450,1421,NaN,71 PRINCE ARTHUR AVE,2008,0
548703,20080314,348,PARK IN DISABLED NO PERMIT,450,1730,NaN,2901 BAYVIEW AV,2008,0


Appears that they stopped fining people for parking without a permit?? or perhaps this is a data issue??

In [60]:
tpt_df[(tpt_df['infraction_code']==130)]

,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,year,year2
257773,20140527,130,STOP PROH TIME MUTUAL STREET,0,1153,RO,2184 EGLINTON AVE W,2014,0


In [61]:
tpt_df[(tpt_df['infraction_code']==256)]

,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,year,year2
914850,20120427,256,LEAVE OVERNIGHT WITHOUT PERMIT,0,1246,NaN,3 MASSEY SQ,2012,0


Looks like most of these $0 fine amt exceptions are pretty rare...

In [64]:
tpt_df_grouped[(tpt_df_grouped['location2']=='393 UNIVERSITY AVE') &(tpt_df_grouped['infraction_code']==15) ]

,year,infraction_code,location2,count
206509,2008,15,393 UNIVERSITY AVE,465
592857,2009,15,393 UNIVERSITY AVE,285
962528,2010,15,393 UNIVERSITY AVE,318
1336055,2011,15,393 UNIVERSITY AVE,557
1706193,2012,15,393 UNIVERSITY AVE,710
1984529,2013,15,393 UNIVERSITY AVE,297
2257274,2014,15,393 UNIVERSITY AVE,478
2633081,2015,15,393 UNIVERSITY AVE,317


In [66]:
fine_amount_dict[15]

{'fine_amt': 100.0, 'fine_descp': 'PARK 3 M OF FIRE HYDRANT'}

Sweet lord this fire hydrant made $71,000 in 2012....maybe after this year they decided to paint the curb red??

In [71]:
tpt_df_grouped[(tpt_df_grouped['location2']=='361 UNIVERSITY AVE') &(tpt_df_grouped['infraction_code']==15)]

,year,infraction_code,location2,count
206231,2008,15,361 UNIVERSITY AVE,246
592604,2009,15,361 UNIVERSITY AVE,173
962277,2010,15,361 UNIVERSITY AVE,188
1335785,2011,15,361 UNIVERSITY AVE,296
1705949,2012,15,361 UNIVERSITY AVE,373
1984403,2013,15,361 UNIVERSITY AVE,113
2257035,2014,15,361 UNIVERSITY AVE,276
2632861,2015,15,361 UNIVERSITY AVE,274


In [78]:
tpt_df.groupby(['year'])['set_fine_amount'].sum()#.reset_index()

year
2008    111153475
2009    105288625
2010    109898240
2011    114183140
2012    111449490
2013     42479400
2014    108971405
2015     99851365
Name: set_fine_amount, dtype: int64

In [79]:
tpt_df.groupby(['year','infraction_code'])['set_fine_amount'].sum()#.reset_index()

year  infraction_code
2008  1                    357090
      2                   1967850
      3                   9786870
      4                    362970
      5                  20747050
      6                   2689110
      7                    965500
      8                   6781080
      9                  10719600
      10                   221700
      11                     9045
      12                     9885
      13                     2310
      14                   187800
      15                  3905310
      16                   636440
      17                    12070
      18                     1070
      19                     1050
      20                      620
      21                     1230
      22                       40
      23                      740
      24                     1890
      25                   173460
      26                     8250
      28                   929340
      29                 13580130
      30                  

General notes:
- apparently hydrant fines changed from 30to100 part way through 2008
- with your current method your multipying all fines by $100 so numbers will be a bit off...


To Do:
- interactive dashboard
- time series graphs based on user selection of location and infraction code (or show multiple infraction codes per graph)
- overall summary (fine sums per year, fine sums per year per infrac_code)